In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip "/content/drive/MyDrive/open.zip"

In [2]:
import os
from datetime import datetime
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import Xception
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

In [3]:
# =========================== Xception + AlexNet ===========================

# https://techblog-history-younghunjo1.tistory.com/261
# https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

TRAIN_FOLDER_PATH = '/content/train/'
TEST_FOLDER_PATH = '/content/test/'
# TRAIN_FOLDER_PATH = '../../open/train/'
# TEST_FOLDER_PATH = '../../open/test/'

EPOCHS = 50
RESIZED_WIDTH = 227
RESIZED_HEIGHT = 227

# 모든 이미지 데이터에 대하여 학습
train_img_generator = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.3, shear_range=0.3, horizontal_flip=True, vertical_flip=True, rescale=1/255.)

# defalut
# shuffle=True,
# batch_size=32,
# color_mode='rgb',
# class_mode='categorical',
train_img_flow = train_img_generator.flow_from_directory(
    TRAIN_FOLDER_PATH, target_size=(RESIZED_WIDTH, RESIZED_HEIGHT))

# 3457개의 이미지 데이터 / batch_size 32 = 109
print(len(train_img_flow))

# for img in train_img_flow:
#     print('img :', img)

def create_model_fn():

    input_tensor = Input(shape=(RESIZED_WIDTH, RESIZED_HEIGHT, 3))

    pretrained_model = Xception(
        input_tensor=input_tensor, include_top=False, weights='imagenet')
    pretrained_output = pretrained_model.output

    x = GlobalAveragePooling2D()(pretrained_output)
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(units=4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(units=19, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    return model

model = create_model_fn()
model.compile(optimizer=SGD(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['acc'])

train_history = model.fit(train_img_flow, epochs=EPOCHS)
loss = train_history.history['loss']
print('loss :', loss)

# 연월일_시간
now = datetime.today().strftime('%Y%m%d_%H%M%S')
model.save(f'model_{now}.h5')

# ==========================================================================


Found 3457 images belonging to 19 classes.
109
83683744/83683744 [==============================] - 5s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 113, 113, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 113, 113, 32  128        ['block1_conv1[0